In [ ]:
from google.colab import files

uploaded = files.upload()

Saving The_Verdict.txt to The_Verdict.txt


In [ ]:
with open("The_Verdict.txt","r",encoding="utf-8") as f:
  raw_text = f.read()

print("Total number of character:",len(raw_text))
print(raw_text[:99])

Total number of character: 20546
The Verdict
Edith Wharton

1908

Exported from Wikisource on May 20, 2024

1

I HAD always thought


In [ ]:
# Separating the word using regular expression library
import re

txt = "Hello . This, is a test "
result = re.split(r'(\s)',txt) #This \s will split spaces wherever white spaces are encountered
print(result)

['Hello', ' ', '.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test', ' ', '']


In [ ]:
# To separate [,.]
result = re.split(r"[,.](\s)",txt)
print(result)

['Hello ', ' ', 'This', ' ', 'is a test ']


As you can see white spaces are also counted as tokens.We'll solve them in the next step

In [ ]:
result = [item for item in result if item.strip()]
print(result)

['Hello ', 'This', 'is a test ']


The tokenisation scheme we divised works well on sample text. But we also need to include question marks,double dashes too.


In [ ]:
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)',text)
result = [item.strip() for item in result if item and item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [ ]:
preprocessed_text = re.split(r'([,.:;?_!"()\']|--|\s)',raw_text)
preprocessed_text = [item.strip() for item in preprocessed_text if item and item.strip()]
print(preprocessed_text[:30])
print(len(preprocessed_text))

['The', 'Verdict', 'Edith', 'Wharton', '1908', 'Exported', 'from', 'Wikisource', 'on', 'May', '20', ',', '2024', '1', 'I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius-though', 'a', 'good', 'fellow', 'enough', '--', 'so']
4707


STEP 2- CREATING TOKEN ID'S

In [ ]:
all_words = sorted(set(preprocessed_text))
vocab_size = len(all_words)
print(vocab_size)

1158


In [ ]:
vocab = {token:integer for integer ,token in enumerate(all_words)}

In [ ]:
for i,item in enumerate(vocab.items()):
  print(item)
  if i>=50:
    break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('-she', 7)
('.', 8)
('1', 9)
('10', 10)
('11', 11)
('12', 12)
('13', 13)
('14', 14)
('15', 15)
('16', 16)
('17', 17)
('1908', 18)
('2', 19)
('20', 20)
('2024', 21)
('3', 22)
('4', 23)
('5', 24)
('6', 25)
('7', 26)
('8', 27)
('9', 28)
(':', 29)
(';', 30)
('?', 31)
('A', 32)
('Ah', 33)
('Among', 34)
('And', 35)
('Are', 36)
('Arrt', 37)
('As', 38)
('At', 39)
('Be', 40)
('Begin', 41)
('Burlington', 42)
('But', 43)
('By', 44)
('Carlo', 45)
('Chicago', 46)
('Claude', 47)
('Come', 48)
('Croft', 49)
('Destroyed', 50)


In [ ]:
class SimpleTokenizerV1:
  def __init__(self,vocab):
    self.str_to_int = vocab
    self.int_to_str = {i:s for s,i in vocab.items()}

  def encode(self,text):
    preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',text)

    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    ids = [self.str_to_int[s] for s in preprocessed]
    return ids


  def decode(self,ids):
    text=" ".join([self.int_to_str[i] for i in ids])
    #Replace spaces before the specified punctuations
    text = re.sub(r'\s+([,.?!"()\'])',r'\1',text)
    return text

In [ ]:
tokenizer = SimpleTokenizerV1(vocab)

text ="""It's the last he painted  , you know, "
          Mrs.Gisburn said with pardonable pride."""

ids = tokenizer.encode(text)
print(ids)

[79, 2, 878, 1016, 629, 558, 773, 5, 1154, 623, 5, 1, 91, 8, 61, 879, 1136, 782, 821, 8]


In [ ]:
tokenizer.decode(ids)

'It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

So far, so good . We implemented a tokenizer capable of tokenizing and de-tokenizing the text . Let's now apply it to the new text that is not contained in the training set.

In [ ]:
text = "Hello , do you like tea"
print(tokenizer.encode(text))

[1159, 5, 380, 1154, 655, 1003]


   
The problem is that the word "Hello" was not used in the The Verdict short story.

Hence, it
is not contained in the vocabulary.

This highlights the need to consider large and diverse
training sets to extend the vocabulary when working on LLMs.


### ADDING SPECIAL CONTEXT TOKENS

In the previous section, we implemented a simple tokenizer and applied it to a passage
from the training set.

In this section, we will modify this tokenizer to handle unknown
words.


In particular, we will modify the vocabulary and tokenizer we implemented in the
previous section, SimpleTokenizerV2, to support two new tokens, <|unk|> and
<|endoftext|>


<div class="alert alert-block alert-warning">

We can modify the tokenizer to use an <|unk|> token if it
encounters a word that is not part of the vocabulary.

Furthermore, we add a token between
unrelated texts.

For example, when training GPT-like LLMs on multiple independent
documents or books, it is common to insert a token before each document or book that
follows a previous text source

</div>




<div class="alert alert-block alert-success">

Let's now modify the vocabulary to include these two special tokens, <unk> and
<|endoftext|>, by adding these to the list of all unique words that we created in the
previous section:
</div>

In [ ]:
all_tokens = sorted(set(preprocessed_text))
all_tokens.extend(["<|endoftext|>","<|unk|>"])
vocab ={token:integer for integer,token in enumerate(all_tokens)}


In [ ]:
print(len(all_tokens))

1160


<div class="alert alert-block alert-info">
    
Based on the output of the print statement above, the new vocabulary size is 1256 (the
vocabulary size in the previous section was 1254).

</div>



<div class="alert alert-block alert-success">

As an additional quick check, let's print the last 5 entries of the updated vocabulary:
</div>

In [ ]:
for i,item in enumerate(list(vocab.items())[-5:]):
  print(item)

('younger', 1155)
('your', 1156)
('yourself', 1157)
('<|endoftext|>', 1158)
('<|unk|>', 1159)


<div class="alert alert-block alert-success">

A simple text tokenizer that handles unknown words</div>



<div class="alert alert-block alert-info">
    
Step 1: Replace unknown words by <|unk|> tokens
    
Step 2: Replace spaces before the specified punctuations

</div>


In [ ]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [ ]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [ ]:
tokenizer.encode(text)


[1159,
 5,
 380,
 1154,
 655,
 1003,
 31,
 1158,
 78,
 1016,
 984,
 1012,
 749,
 1016,
 1159,
 8]

In [ ]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'


<div class="alert alert-block alert-info">
    
Based on comparing the de-tokenized text above with the original input text, we know that
the training dataset, Edith Wharton's short story The Verdict, did not contain the words
"Hello" and "palace."

</div>


<div class="alert alert-block alert-warning">

So far, we have discussed tokenization as an essential step in processing text as input to
LLMs. Depending on the LLM, some researchers also consider additional special tokens such
as the following:


[BOS] (beginning of sequence): This token marks the start of a text. It
signifies to the LLM where a piece of content begins.


[EOS] (end of sequence): This token is positioned at the end of a text,
and is especially useful when concatenating multiple unrelated texts,
similar to <|endoftext|>. For instance, when combining two different
Wikipedia articles or books, the [EOS] token indicates where one article
ends and the next one begins.


[PAD] (padding): When training LLMs with batch sizes larger than one,
the batch might contain texts of varying lengths. To ensure all texts have
the same length, the shorter texts are extended or "padded" using the

[PAD] token, up to the length of the longest text in the batch.

</div>


<div class="alert alert-block alert-warning">

Note that the tokenizer used for GPT models does not need any of these tokens mentioned
above but only uses an <|endoftext|> token for simplicity

</div>

<div class="alert alert-block alert-warning">

the tokenizer used for GPT models also doesn't use an <|unk|> token for outof-vocabulary words. Instead, GPT models use a byte pair encoding tokenizer, which breaks
down words into subword units
</div>

# **BYTE PAIR ENCODING**

**BPE TOKENIZER**

We will use the library tiktoken which efficiently implements BPE algoritgm in rust

In [ ]:
 ! pip3 install tiktoken

In [ ]:
import importlib
import tiktoken

print("tiktoken version: ", importlib.metadata.version("tiktoken"))

tiktoken version:  0.11.0


In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
text=(
     "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integer = tokenizer.encode(text,allowed_special={"<|endoftext|>"})
print(integer)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [ ]:
strings = tokenizer.decode(integer)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


In [ ]:
integers = tokenizer.encode("Akwirw ier")
print(integers)

strings = tokenizer.decode(integers)
print(strings)

[33901, 86, 343, 86, 220, 959]
Akwirw ier


**Creating an input target pair**

In this section we implement a Datal loader that fetches the input-target pairs using a sliding window approach.

In [ ]:
enc_text = tokenizer.encode(raw_text)

In [ ]:
enc_sample = enc_text[50:]

One of the easiest and most intuitive ways to create the input-target pairs for the nextword prediction task is to create two variables ,x and y where x contains the input tokens and y contains the target,which are the inputs shifted by 1:

The context size determines how many tokens are included in the input

In [ ]:
context_size = 4 #length of the input
# The context_size of 4 means   that the model is trained to look at a sequence of 4 words (or tokens) to predict the next word in the sequence
# The input x is the first 4 tokens [1,2,3,4] and the target y is the next 4 tokens [2,3,4,5]

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x:{x}")
print(f"y:     {y}")

x:[568, 340, 373, 645]
y:     [340, 373, 645, 1049]


Processing the inputs along with the targets,which are inputs shifted by one position, we can then create the next word predicton task as follows

In [ ]:
for i in range(1,context_size+1):
  context = enc_sample[:i]
  desired = enc_sample[i]

  print(context , "-->" ,desired)

[568] --> 340
[568, 340] --> 373
[568, 340, 373] --> 645
[568, 340, 373, 645] --> 1049


In [ ]:
for i in range(1,context_size+1):
  context = enc_sample[:i]
  desired = enc_sample[i]

  print(tokenizer.decode(context) , "-->" , tokenizer.decode([desired]))

so -->  it
so it -->  was
so it was -->  no
so it was no -->  great


**Implementing Data Loader**

For the efficient data loader implementation, we will use Pytorch's built in Dataset and DataLoader classes.

In [ ]:
from torch.utils.data import Dataset,DataLoader
import torch

class GPTDatasetV1(Dataset):
  def __init__(self,txt,tokenizer,max_length,stride):
    self.input_ids=[]
    self.token_ids =[]

    #tokenize the entire the text
    token_ids = tokenizer.encode(txt,allowed_special={"<|endoftext|>"})

    #Use a sliding window to chunk the book into overlapping sequences of max_length
    for i in range (0,len(token_ids)-max_length,stride):
      input_chunk =token_ids[i:i+max_length]
      target_chunk = token_ids[i+1:i+max_length+1]
      self.input_ids.append(torch.tensor(input_chunk))
      self.token_ids.append(torch.tensor(target_chunk))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self,idx):   # here idx refers to row , if idex is 0 it returns the first row of the input tensor and first row of the output tensor
    return self.input_ids[idx],self.token_ids[idx]

The GPTDatasetV1 class is listing 2.5 is based on the Pytorch Dataset class.


It defines how individual rows are fetched from the dataset.


Each row consists of number of token ID's (based on a max_length) assigned to input_chunk tensor.

The target_chunk tensor contains the corresponding targets.


Step 1:Initialize the tokenizer

Step 2: Create the Dataset

Step 3: drop_last = True drops the last batch if it is shorter than the specified batch_size to preven loss spiked during training

Step 4: The number of CPU  processes to use for preprocessing

In [ ]:
  def create_dataloader_v1(txt, batch_size=4, max_length=256,                #batch size =number of threads running on our cpu parallelly
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [ ]:
with open("The_Verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [ ]:
import torch
print("PyTorch version:", torch.__version__)
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

PyTorch version: 2.8.0+cu126
[tensor([[  464,  4643, 11600,   198]]), tensor([[ 4643, 11600,   198,  7407]])]


The first_batch contains two tensors: the first tensors stores the input token ID's and the second tensors stores the target token ID's

Since the max_length is set to 4 , each of the two tensors contains 4 token Id's

Note that the input size of 4 is relatively small and only chosen for illustration purposes. It is common to train the LLM with input size of at least 256


The stride setting dictates the number of positions the inputs shifts across batches ,emulating a sliding window approach

In [ ]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[  464,  4643, 11600,   198],
        [ 7407,   342,   854, 41328],
        [  198,   198,  1129,  2919],
        [  198,   198,  3109,  9213],
        [  422, 11145,   271,  1668],
        [  319,  1737,  1160,    11],
        [48609,   198,   198,    16],
        [  628,   200,    40,   367]])

Targets:
 tensor([[ 4643, 11600,   198,  7407],
        [  342,   854, 41328,   198],
        [  198,  1129,  2919,   198],
        [  198,  3109,  9213,   422],
        [11145,   271,  1668,   319],
        [ 1737,  1160,    11, 48609],
        [  198,   198,    16,   628],
        [  200,    40,   367,  2885]])


**Creating Token Embeddings**

Lets' illustrate how the token ID to embedding vector conversion works with hands-on example. Suppose we have the following four input tokens with ID's 2,3,5,1:

In [ ]:
input_ids = torch.tensor([2,3,5,1])

For the sake of simplicity ,suppose we have a small vocabulary of only 6 words (instead of 50257 words in the BPE tokenizer voacabulary), and we want to create embeddings of size 3 (int GPT 3 ,the embedding size is 12288 dimensions )

Using the vocab_size and output_dim,we can instantiate an embedding layer in Pytorch ,setting the random seed to 123 for reproducibilty purpose:

In [ ]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size,output_dim)  #torch.nn.Embedding is simple lookup table that stores embeddings of fixed dictionary and sizes

The print statement in the code prints the embedding layer's underlying weight matrix

In [ ]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


We can see that the weight matrix of the embedding layer contains,small and random values . These values are optimized during LLM training as part of the LLM optimization itself, as we will see in upcoming chapters.Moreover, we can see that the weight has six rows and three columns . There is one row for each of the six possible tokens in the vocabulary . And there is one column for each of the three embedding dimensions.

After we instantiated the embedding layer ,let's now apply it to a token ID to obtain embedding vector

In [ ]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


**POSITIONAL EMBEDDING**(ENCODING WORD POSITIONS)

Previously, we focussed on very small embedding sizes in this chapter .
We now consider more realistic and useful embedding sizes and encode the input tokens into a 256 dimensional vector representation.


In [ ]:
vocab_size = 50257
output_dim = 256

token_embedding_layer= torch.nn.Embedding(vocab_size,output_dim)

In [ ]:
max_length = 4
dataloader = create_dataloader_v1(raw_text,batch_size=8,max_length=max_length,stride=max_length,shuffle=False)  #batch_size = 8 means 8 text sample  or input sequences with 4 token
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

Data_loader helps us to manage the task of managing the batching the data ,inputing the data ,making parallel processing easier

In [ ]:
print("Token_IDs:\n", inputs)
print("\nInputs shape:\n",inputs.shape)

Token_IDs:
 tensor([[  464,  4643, 11600,   198],
        [ 7407,   342,   854, 41328],
        [  198,   198,  1129,  2919],
        [  198,   198,  3109,  9213],
        [  422, 11145,   271,  1668],
        [  319,  1737,  1160,    11],
        [48609,   198,   198,    16],
        [  628,   200,    40,   367]])

Inputs shape:
 torch.Size([8, 4])


As we can see, the token Id tensor is 8x4 dimensional ,meaning that the data batch consists of 8 text sample with 4 tokens each

Let's now use the embedding layer to embed these token IDs into 256 dimensional vectors

In [ ]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


For a GPT model ,Absolute embedding approach , we just need to create another embedding layer that has the same  dimension as the token_ID

In [ ]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length,output_dim)

In [ ]:
pos_embedding = pos_embedding_layer(torch.arange(max_length))
print(pos_embedding.shape)

torch.Size([4, 256])


In [ ]:
input_embeddings = token_embeddings + pos_embedding
print(input_embeddings.shape)

torch.Size([8, 4, 256])
